In [5]:
%run ./init.ipynb

Length of attributes added  100000
Number of valid rows with attributes  97040


In [6]:
missingIDs = np.where(pd.isnull(df['Background']))[0]
print(missingIDs)

[97040 97041 97042 ... 99997 99998 99999]


In [7]:
listing = {}

In [ ]:
%%time
startRange = missingIDs[0]
stopRange = 100001

refStartx = startRange
cookies = {}
#for x in range(startRange, stopRange):
for x in ([97040]):
    scraper = cloudscraper.create_scraper(debug=True,
        delay=1,
        browser={
            'browser': 'firefox',
            'platform': 'windows',
            'mobile': False
        })
    #time.sleep(10)
    #print(scraper.get_cookie_string("https://niftys.com/nft/0x39ceaa47306381b6d79ad46af0f36bc5332386f2/1"))
    
    user_agent = scraper.get_cookie_string('https://niftys.com/thematrix')
    
    #html = scraper.get("https://niftys.com/thematrix").text
    print(user_agent)
    

    break
   # session = requests.Session()
   # session.headers = {'cf_clearance': '079CebCgFDHqfxuqJB6fvbdWJlZlICQ.ihLNGddPbSI-1641486694-0-150'}

    #html = scraper.get("https://niftys.com/nft/0x39ceaa47306381b6d79ad46af0f36bc5332386f2/" + str(x + 1) + "/").text
    html = scraper.get("https://niftys.com/nft/0x39ceaa47306381b6d79ad46af0f36bc5332386f2").text
   
    print(html)

    break
    soup = BeautifulSoup(html, 'lxml')

    rows = soup.find_all('div', 'label')

    rowAttributes = {}
    for row in rows:
        cells = row
        str_row = str(row)
        value = str_row.split('!-- -->')
        cleanTextAttribute = BeautifulSoup(value[0], "lxml").get_text()
        if cleanTextAttribute == 'Available on PALM':
            continue
        if (len(value) > 1):
            cleanTextValue = BeautifulSoup(value[1], "lxml").get_text().strip()
        else:
            cleanTextValue = ""
        
        rowAttributes[cleanTextAttribute] = cleanTextValue
    
    output_list = [(rowAttributes[key]) for key in (manualColumns) if key in rowAttributes]

    print(output_list)
    listing[str(x+1)] = output_list
    #clear_output(wait=True)
    print(x+1)
# print(listing)
# data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
# data_dictionary.to_csv('MissingAttributes_' + str(missingIDs[0]) + '_' + str(missingIDs[len(missingIDs)-1]) + '.csv')
# refStartx = x+1
# listing.clear()

ERROR:root:"https://niftys.com/thematrix" returned an error. Could not collect tokens.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://niftys.com/thematrix

During handling of the above exception, another exception occurred:

AttributeError: 'HTTPError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://niftys.com/thematrix

During handling of the above exception, another exception occurred:

AttributeError: 'HTTPError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

TypeError: object of type 'NoneType' has no len()

During handling of the above exception, another exception occurred:

AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://niftys.com/

In [ ]:
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
data_dictionary.to_csv('MissingAttributes_' + str(missingIDs[0]) + '_' + str(missingIDs[len(missingIDs)-1]) + '.csv')

In [ ]:
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index')  #pd.DataFrame(listing, orient='index')

In [ ]:
for count in range(0, len(manualColumns)):
    print(count)
    print(data_dictionary[manualColumns[count]].value_counts())
    data_dictionary[manualColumns[count]].value_counts(normalize=True)
    print('---------------------------------')

In [ ]:
print('done')